In [1]:
from tensorflow.contrib.learn.python.learn.datasets import base

import tensorflow as tf
import numpy as np
import os,sys
sys.path.insert(0, './scripts')
dataDir ='./data'


import py_compile
py_compile.compile('scripts/ivector_dataset.py')
py_compile.compile('scripts/siamese_model.py')
import ivector_dataset
import siamese_model
import ivector_tools as it

In [2]:
def get_dataset_size(dict_file,feat_file):
# Counting feature dimension and total number of utterances
    f = open(dict_file)
    dict_dim = 0
    for line in f:
        dict_dim+=1
    f.close()
    feat_len = 0
    f = open(feat_file)
    for line in f:
        feat_len+=1
    f.close()
    return dict_dim, feat_len

def get_feat_label(dict_file, feat_file):
# Get feature vectors from files
    dict_dim, feat_len = get_dataset_size(dict_file,feat_file)
    features = np.zeros((feat_len,dict_dim),dtype='float32')
    labels = np.zeros((feat_len),dtype='int8')
    names = []
    f = open(feat_file)
    count = 0
    for line in f:
        names.append(line.split()[0])
        labels[count] = line.split()[1]
        line= line.split()[2:]
        for iter in range(0,len(line)):
            elements = line[iter].split(':')
            features[count][ int( elements[0] ) -1 ] = elements[1]
        count = count + 1 
    f.close()
    
    return features, labels, names

In [3]:
# write prototxt for siamese network

languages = ['EGY','GLF','LAV','MSA','NOR']
trn_labels = []
trn_names = []
trn_ivectors = np.empty((0,400))
dev_labels = []
dev_names = []
dev_ivectors = np.empty((0,400))


for i,lang in enumerate(languages):
    #load train.vardial2017
    filename = dataDir+'/train.vardial2017/%s.ivec' % lang
    name   = np.loadtxt(filename,usecols=[0],dtype='string')
    ivector = np.loadtxt(filename,usecols=range(1,401),dtype='float32')
    trn_labels = np.append(trn_labels, np.ones(np.size(name))*(i+1))
    trn_names=np.append(trn_names,name)
    trn_ivectors = np.append(trn_ivectors, ivector,axis=0)

    #load dev.vardial2017
    filename = dataDir+'/dev.vardial2017/%s.ivec' % lang
    name   = np.loadtxt(filename,usecols=[0],dtype='string')
    ivector = np.loadtxt(filename,usecols=range(1,401),dtype='float32')
    dev_names=np.append(dev_names,name)
    dev_ivectors = np.append(dev_ivectors, ivector,axis=0)
    dev_labels = np.append(dev_labels, np.ones(np.size(name))*(i+1))
    
# load test.MGB3
filename = dataDir+'/test.MGB3/ivec_features'
tst_names   = np.loadtxt(filename,usecols=[0],dtype='string')
tst_ivectors = np.loadtxt(filename,usecols=range(1,401),dtype='float32')

# merge trn+dev
trndev_ivectors = np.append(trn_ivectors, dev_ivectors,axis=0)
trndev_labels = np.append(trn_labels,dev_labels)
trndev_name = np.append(trn_names,dev_names)


# load tst.MGB3 labels
filename = 'data/test.MGB3/reference'
tst_ref_names = np.loadtxt(filename,usecols=[0],dtype='string')
tst_ref_labels = np.loadtxt(filename,usecols=[1],dtype='int')

tst_ref_labels_index = []
for i, name_ref in enumerate(tst_ref_names):
    for j, name in enumerate(tst_names):
        if name == name_ref:
            tst_ref_labels_index = np.append(tst_ref_labels_index,int(j))

tst_labels = tst_ref_labels
tst_ivectors = tst_ivectors[ map(int,tst_ref_labels_index),:]

#center and length norm.
m=np.mean(trn_ivectors,axis=0)
A = np.cov(trn_ivectors.transpose())
[a,D,V] = np.linalg.svd(A)
V= V.transpose()
W= np.dot(V, np.diag(1./( np.sqrt(D) + 0.0000000001 )))

trn_ivectors = np.dot( np.subtract( trn_ivectors, m), W)
trndev_ivectors = np.dot( np.subtract( trndev_ivectors, m), W)
dev_ivectors = np.dot( np.subtract( dev_ivectors, m), W)
tst_ivectors = np.dot( np.subtract( tst_ivectors, m), W)

trn_ivectors = it.length_norm(trn_ivectors)
trndev_ivectors = it.length_norm(trndev_ivectors)
dev_ivectors = it.length_norm(dev_ivectors)
tst_ivectors = it.length_norm(tst_ivectors)


#language modeling
lang_mean=[]
for i, lang in enumerate(languages):
    lang_mean.append(np.mean(np.append(trn_ivectors[np.nonzero(trn_labels == i+1)] ,9*dev_ivectors[np.nonzero(dev_labels == i+1)],axis=0),axis=0))
#     lang_mean.append(np.mean(dev_ivectors[np.nonzero(dev_labels == i+1)],axis=0))

lang_mean = np.array(lang_mean)
lang_mean = it.length_norm(lang_mean)

print( np.shape(trn_ivectors), np.shape(dev_ivectors), np.shape(lang_mean),np.shape(tst_ivectors) )


# Baseline performance on TST using CDS
tst_scores = lang_mean.dot(tst_ivectors.transpose() )
# print(tst_scores.shape)
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))
print 'Baseline accurary on test dataset : %0.3f' %(acc)

tst_scores_ivector_baseline = tst_scores

((13825, 400), (1524, 400), (5, 400), (1492, 400))
Baseline accurary on test dataset : 0.668


In [4]:
# init variables
sess = tf.InteractiveSession()
siamese = siamese_model.siamese();
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.005, global_step,
                                           5000, 0.99, staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(siamese.loss, global_step=global_step)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


saver_folder='snnmodel_ivector_backup'
# saver_folder='snnmodel_ivector'

max_step = 42320
# max_step = 51240

print max_step
RESTORE_STEP=max_step
saver.restore(sess, saver_folder+'/model'+str(RESTORE_STEP)+'.ckpt')


# trn_ivectors_siam = siamese.o1.eval({siamese.x1:trn_ivectors})
dev_ivectors_siam = siamese.o1.eval({siamese.x1:dev_ivectors})
tst_ivectors_siam = siamese.o1.eval({siamese.x1:tst_ivectors})
lang_mean_siam = siamese.o1.eval({siamese.x1:lang_mean})

tst_scores = lang_mean_siam.dot(tst_ivectors_siam.transpose() )
# print(tst_scores.shape)
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))
print 'Final accurary on test dataset : %0.3f' %(acc)

confusionmat = np.zeros((5,5))
for i,lang in enumerate(languages):
    hypo_bylang = hypo_lang[ tst_labels == i+1]
    hist_bylang = np.histogram(hypo_bylang,5)
    confusionmat[:,i] = hist_bylang[0]

precision = np.diag(confusionmat) / np.sum(confusionmat,axis=1) #precision
recall = np.diag(confusionmat) / np.sum(confusionmat,axis=0) # recall
    
print 'Confusion matrix'
print confusionmat
print 'Precision'
print precision
print 'Recall'
print recall

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))

sess.close()
tf.reset_default_graph()
tst_scores_ivectors = tst_scores

(?, 16000)
(?, 1500)
(?, 600)
(?, 16000)
(?, 1500)
(?, 600)
42320
INFO:tensorflow:Restoring parameters from snnmodel_ivector_backup/model42320.ckpt
Final accurary on test dataset : 0.731
Confusion matrix
[[ 225.   13.   37.   11.   24.]
 [  11.  178.   31.   16.   12.]
 [  43.   45.  212.   14.   31.]
 [   5.    5.   10.  208.   10.]
 [  18.    9.   44.   13.  267.]]
Precision
[ 0.72580645  0.71774194  0.61449275  0.87394958  0.76068376]
Recall
[ 0.74503311  0.712       0.63473054  0.79389313  0.77616279]


<Performance evaluation on Test dataset>
Accurary  : 0.731
Precision : 0.739
Recall    : 0.732


In [5]:
context = 1
dict_file = 'data/train.vardial2017/dict.words.c'+str(context)
feat_file = 'data/train.vardial2017/words.c'+str(context)
trn_features, trn_labels, trn_names = get_feat_label(dict_file,feat_file)

feat_file = 'data/dev.vardial2017/words.c'+str(context)
dev_features, dev_labels, dev_names = get_feat_label(dict_file,feat_file)

feat_file = 'data/test.MGB3/words.c'+str(context)
tst_features, tst_labels, tst_names = get_feat_label(dict_file,feat_file)

print trn_features.shape, dev_features.shape, tst_features.shape

languages = ['EGY','GLF','LAV','MSA','NOR']

# load tst.MGB3 labels
filename = 'data/test.MGB3/reference'
tst_ref_names = np.loadtxt(filename,usecols=[0],dtype='string')
tst_ref_labels = np.loadtxt(filename,usecols=[1],dtype='int')

tst_ref_labels_index = []
for i, name_ref in enumerate(tst_ref_names):
    for j, name in enumerate(tst_names):
        if name == name_ref:
            tst_ref_labels_index = np.append(tst_ref_labels_index,int(j))

tst_labels = tst_ref_labels
tst_features = tst_features[ map(int,tst_ref_labels_index),:]
    
#language modeling
lang_mean=[]
for i, lang in enumerate(languages):
   lang_mean.append(np.mean( trn_features[np.nonzero(trn_labels == i+1)][:],axis=0 ) )
#     lang_mean.append(np.mean(np.append(trn_features[np.nonzero(trn_labels == i+1)] ,9*dev_features[np.nonzero(dev_labels == i+1)],axis=0),axis=0))

lang_mean = np.array(lang_mean)
lang_mean = it.length_norm(lang_mean)

print( np.shape(trn_features), np.shape(dev_features), np.shape(lang_mean),np.shape(tst_features) )



(14000, 41657) (1524, 41657) (1492, 41657)
((14000, 41657), (1524, 41657), (5, 41657), (1492, 41657))


In [6]:
import siamese_model_words as siamese_model

# init variables
sess = tf.InteractiveSession()
siamese = siamese_model.siamese(np.shape(trn_features)[1]);
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.01, global_step,
                                           5000, 0.99, staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(siamese.loss, global_step=global_step)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
saver_folder='snnmodel_words'
max_step = 60400
print max_step
RESTORE_STEP=max_step
saver.restore(sess, saver_folder+'/model'+str(RESTORE_STEP)+'.ckpt')


# trn_features_siam = siamese.o1.eval({siamese.x1:trn_features})
# dev_features_siam = siamese.o1.eval({siamese.x1:dev_features})
tst_features_siam = siamese.o1.eval({siamese.x1:tst_features})
lang_mean_siam = siamese.o1.eval({siamese.x1:lang_mean})

tst_scores = lang_mean_siam.dot(tst_features_siam.transpose() )
# print(tst_scores.shape)
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))
print 'Final accurary on test dataset : %0.3f' %(acc)

confusionmat = np.zeros((5,5))
for i,lang in enumerate(languages):
    hypo_bylang = hypo_lang[ tst_labels == i+1]
    hist_bylang = np.histogram(hypo_bylang,5)
    confusionmat[:,i] = hist_bylang[0]

precision = np.diag(confusionmat) / np.sum(confusionmat,axis=1) #precision
recall = np.diag(confusionmat) / np.sum(confusionmat,axis=0) # recall
    
print 'Confusion matrix'
print confusionmat
print 'Precision'
print precision
print 'Recall'
print recall

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))

sess.close()
tf.reset_default_graph()
tst_scores_words = tst_scores

(?, 92600)
(?, 1500)
(?, 600)
(?, 92600)
(?, 1500)
(?, 600)
60400
INFO:tensorflow:Restoring parameters from snnmodel_words/model60400.ckpt
Final accurary on test dataset : 0.585
Confusion matrix
[[ 179.   31.   38.   15.   29.]
 [  24.  119.   44.   28.   56.]
 [  54.   41.  208.   23.   47.]
 [  16.   31.   17.  184.   29.]
 [  29.   28.   27.   12.  183.]]
Precision
[ 0.6130137   0.43911439  0.55764075  0.66425993  0.65591398]
Recall
[ 0.59271523  0.476       0.62275449  0.70229008  0.53197674]


<Performance evaluation on Test dataset>
Accurary  : 0.585
Precision : 0.586
Recall    : 0.585


In [7]:
import siamese_model_chars as siamese_model

context = 3
dict_file = 'data/train.vardial2017/dict.chars.c'+str(context)
feat_file = 'data/train.vardial2017/chars.c'+str(context)
trn_features, trn_labels, trn_names = get_feat_label(dict_file,feat_file)

feat_file = 'data/dev.vardial2017/chars.c'+str(context)
dev_features, dev_labels, dev_names = get_feat_label(dict_file,feat_file)

feat_file = 'data/test.MGB3/chars.c'+str(context)
tst_features, tst_labels, tst_names = get_feat_label(dict_file,feat_file)

print trn_features.shape, dev_features.shape, tst_features.shape

languages = ['EGY','GLF','LAV','MSA','NOR']

# load tst.MGB3 labels
filename = 'data/test.MGB3/reference'
tst_ref_names = np.loadtxt(filename,usecols=[0],dtype='string')
tst_ref_labels = np.loadtxt(filename,usecols=[1],dtype='int')

tst_ref_labels_index = []
for i, name_ref in enumerate(tst_ref_names):
    for j, name in enumerate(tst_names):
        if name == name_ref:
            tst_ref_labels_index = np.append(tst_ref_labels_index,int(j))

tst_labels = tst_ref_labels
tst_features = tst_features[ map(int,tst_ref_labels_index),:]


#language modeling
lang_mean=[]
for i, lang in enumerate(languages):
    lang_mean.append(np.mean( trn_features[np.nonzero(trn_labels == i+1)][:],axis=0 ) )

lang_mean = np.array(lang_mean)
lang_mean = it.length_norm(lang_mean)

print( np.shape(trn_features), np.shape(dev_features), np.shape(lang_mean),np.shape(tst_features) )

(14000, 13285) (1524, 13285) (1492, 13285)
((14000, 13285), (1524, 13285), (5, 13285), (1492, 13285))


In [8]:
# init variables
sess = tf.InteractiveSession()
siamese = siamese_model.siamese(np.shape(trn_features)[1]);
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.01, global_step,
                                           5000, 0.99, staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(siamese.loss, global_step=global_step)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
saver_folder='snnmodel_chars'

max_step = 71600
print max_step
RESTORE_STEP=max_step
saver.restore(sess, saver_folder+'/model'+str(RESTORE_STEP)+'.ckpt')


# trn_features_siam = siamese.o1.eval({siamese.x1:trn_features})
# dev_features_siam = siamese.o1.eval({siamese.x1:dev_features})
tst_features_siam = siamese.o1.eval({siamese.x1:tst_features})
lang_mean_siam = siamese.o1.eval({siamese.x1:lang_mean})

tst_scores = lang_mean_siam.dot(tst_features_siam.transpose() )
# print(tst_scores.shape)
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))
print 'Final accurary on test dataset : %0.3f' %(acc)

confusionmat = np.zeros((5,5))
for i,lang in enumerate(languages):
    hypo_bylang = hypo_lang[ tst_labels == i+1]
    hist_bylang = np.histogram(hypo_bylang,5)
    confusionmat[:,i] = hist_bylang[0]

precision = np.diag(confusionmat) / np.sum(confusionmat,axis=1) #precision
recall = np.diag(confusionmat) / np.sum(confusionmat,axis=0) # recall
    
print 'Confusion matrix'
print confusionmat
print 'Precision'
print precision
print 'Recall'
print recall

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))

sess.close()
tf.reset_default_graph()
tst_scores_chars = tst_scores

(?, 88600)
(?, 1500)
(?, 600)
(?, 88600)
(?, 1500)
(?, 600)
71600
INFO:tensorflow:Restoring parameters from snnmodel_chars/model71600.ckpt
Final accurary on test dataset : 0.582
Confusion matrix
[[ 170.   32.   32.   11.   34.]
 [  30.  108.   30.   24.   35.]
 [  57.   44.  214.   24.   50.]
 [  17.   32.   16.  175.   24.]
 [  28.   34.   42.   28.  201.]]
Precision
[ 0.609319    0.47577093  0.55012853  0.66287879  0.6036036 ]
Recall
[ 0.56291391  0.432       0.64071856  0.66793893  0.58430233]


<Performance evaluation on Test dataset>
Accurary  : 0.582
Precision : 0.580
Recall    : 0.578


In [9]:
import siamese_model_phone_hu as siamese_model

context = 3
dict_file = 'data/train.vardial2017/dict.phone_hu.c'+str(context)
feat_file = 'data/train.vardial2017/phone_hu.c'+str(context)
trn_features, trn_labels, trn_names = get_feat_label(dict_file,feat_file)

feat_file = 'data/dev.vardial2017/phone_hu.c'+str(context)
dev_features, dev_labels, dev_names = get_feat_label(dict_file,feat_file)

feat_file = 'data/test.MGB3/phone_hu.c'+str(context)
tst_features, tst_labels, tst_names = get_feat_label(dict_file,feat_file)

print trn_features.shape, dev_features.shape, tst_features.shape

languages = ['EGY','GLF','LAV','MSA','NOR']

# load tst.MGB3 labels
filename = 'data/test.MGB3/reference'
tst_ref_names = np.loadtxt(filename,usecols=[0],dtype='string')
tst_ref_labels = np.loadtxt(filename,usecols=[1],dtype='int')

tst_ref_labels_index = []
for i, name_ref in enumerate(tst_ref_names):
    for j, name in enumerate(tst_names):
        if name == name_ref:
            tst_ref_labels_index = np.append(tst_ref_labels_index,int(j))

tst_labels = tst_ref_labels
tst_features = tst_features[ map(int,tst_ref_labels_index),:]
    
    
#language modeling
lang_mean=[]
for i, lang in enumerate(languages):
#     lang_mean.append(np.mean(np.append(trn_features[np.nonzero(trn_labels == i+1)] ,dev_features[np.nonzero(dev_labels == i+1)],axis=0),axis=0))
    lang_mean.append(np.mean( trn_features[np.nonzero(trn_labels == i+1)][:],axis=0 ) )

lang_mean = np.array(lang_mean)
lang_mean = it.length_norm(lang_mean)

print( np.shape(trn_features), np.shape(dev_features), np.shape(lang_mean),np.shape(tst_features) )

(13825, 50320) (1524, 50320) (1492, 50320)
((13825, 50320), (1524, 50320), (5, 50320), (1492, 50320))


In [10]:
# init variables
sess = tf.InteractiveSession()
siamese = siamese_model.siamese(np.shape(trn_features)[1]);
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.01, global_step,
                                           5000, 0.99, staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(siamese.loss, global_step=global_step)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
saver_folder='snnmodel_phone_hu_backup'

max_step = 60400
print max_step
RESTORE_STEP=max_step
saver.restore(sess, saver_folder+'/model'+str(RESTORE_STEP)+'.ckpt')


# trn_features_siam = siamese.o1.eval({siamese.x1:trn_features})
# dev_features_siam = siamese.o1.eval({siamese.x1:dev_features})
tst_features_siam = siamese.o1.eval({siamese.x1:tst_features})
lang_mean_siam = siamese.o1.eval({siamese.x1:lang_mean})

tst_scores = lang_mean_siam.dot(tst_features_siam.transpose() )
# print(tst_scores.shape)
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))
print 'Final accurary on test dataset : %0.3f' %(acc)

confusionmat = np.zeros((5,5))
for i,lang in enumerate(languages):
    hypo_bylang = hypo_lang[ tst_labels == i+1]
    hist_bylang = np.histogram(hypo_bylang,5)
    confusionmat[:,i] = hist_bylang[0]

precision = np.diag(confusionmat) / np.sum(confusionmat,axis=1) #precision
recall = np.diag(confusionmat) / np.sum(confusionmat,axis=0) # recall
    
print 'Confusion matrix'
print confusionmat
print 'Precision'
print precision
print 'Recall'
print recall

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))

sess.close()
tf.reset_default_graph()
tst_scores_phone_hu = tst_scores

(?, 91520)
(?, 1500)
(?, 600)
(?, 91520)
(?, 1500)
(?, 600)
60400
INFO:tensorflow:Restoring parameters from snnmodel_phone_hu_backup/model60400.ckpt
Final accurary on test dataset : 0.548
Confusion matrix
[[ 170.   34.   73.   19.   50.]
 [  25.  132.   62.   21.   27.]
 [  46.   55.  137.   19.   51.]
 [  17.   11.   15.  187.   25.]
 [  44.   18.   47.   16.  191.]]
Precision
[ 0.49132948  0.49438202  0.44480519  0.73333333  0.60443038]
Recall
[ 0.56291391  0.528       0.41017964  0.71374046  0.55523256]


<Performance evaluation on Test dataset>
Accurary  : 0.548
Precision : 0.554
Recall    : 0.554


In [11]:
# Fusion 1 : ivector + chars

tst_scores = tst_scores_ivectors + tst_scores_chars
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))




<Performance evaluation on Test dataset>
Accurary  : 0.755
Precision : 0.554
Recall    : 0.554


In [12]:
# Fusion 2 : ivector + words

tst_scores = tst_scores_ivectors + tst_scores_words
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))




<Performance evaluation on Test dataset>
Accurary  : 0.751
Precision : 0.554
Recall    : 0.554


In [13]:
# Fusion 3 : ivector + phone_hu

tst_scores = tst_scores_ivectors + tst_scores_phone_hu
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))




<Performance evaluation on Test dataset>
Accurary  : 0.716
Precision : 0.554
Recall    : 0.554


In [14]:
# Fusion 4 : All

tst_scores = 1*tst_scores_ivectors + 1*tst_scores_words + 1*tst_scores_chars + 1*tst_scores_phone_hu
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))




<Performance evaluation on Test dataset>
Accurary  : 0.752
Precision : 0.554
Recall    : 0.554


In [15]:
# Fusion - : All + linear combination

tst_scores = 2.5*tst_scores_ivectors + 1*tst_scores_words + 1*tst_scores_chars + 1*tst_scores_phone_hu
hypo_lang = np.argmax(tst_scores,axis = 0)
temp = ((tst_labels-1) - hypo_lang)
acc =1- np.size(np.nonzero(temp)) / float(np.size(tst_labels))

print '\n\n<Performance evaluation on Test dataset>'
print 'Accurary  : %0.3f' %(acc)
print 'Precision : %0.3f' %(np.mean(precision))
print 'Recall    : %0.3f' %(np.mean(recall))




<Performance evaluation on Test dataset>
Accurary  : 0.775
Precision : 0.554
Recall    : 0.554
